In [3]:
import pandas as pd
import os

# Use the mapping dictionary imported from the teams.csv file to change all full team names to abbreviations
abbreviations_df = pd.read_csv('c:/Users/luken/Desktop/Betting_Model_24_25/teams.csv')
mapping_dict = dict(zip(abbreviations_df['Team'], abbreviations_df['Abbrv']))

# Mapping function
def update_and_overwrite_file(file_path, mapping_dict):
    df = pd.read_csv(file_path)
    df['Team'] = df['Team'].map(mapping_dict)
    df.to_csv(file_path, index=False)  # Overwrite the original file

# Directory containing the raw data
input_directory = "c:/Users/luken/Desktop/Betting_Model_24_25/Raw_games_training"

# Collect all file paths from the directory
file_paths = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith(".csv")]

# Loop through the files and apply the mapping function
for file_path in file_paths:
    update_and_overwrite_file(file_path, mapping_dict)

print("Team names updated successfully in all files.")

Team names updated successfully in all files.


In [4]:
import pandas as pd
import re
import os

# Use the mapping dictionary to change all short team names to abbreviations
abbreviations_df = pd.read_csv('c:/Users/luken/Desktop/Betting_Model_24_25/teams.csv')  # Adjusted path for teams.csv
mapping_dict = dict(zip(abbreviations_df['Name'], abbreviations_df['Abbrv']))

# Function to replace team names in the "Game" column using regex
def replace_team_names(game_string, mapping_dict):
    pattern = r' (\d{4}-\d{2}-\d{2}) - ([\w\s]+) (\d+), ([\w\s]+) (\d+)'
    match = re.match(pattern, game_string)

    if match:
        date, team1, score1, team2, score2 = match.groups()
        team1_abbr = mapping_dict.get(team1.strip(), team1)  # Fallback to original if not found
        team2_abbr = mapping_dict.get(team2.strip(), team2)  # Fallback to original if not found

        return f"{date} - {team1_abbr} {score1}, {team2_abbr} {score2}"
    else:
        return game_string 

# Function to update the "Game" column and overwrite the file
def update_and_overwrite_file(file_path, mapping_dict):
    df = pd.read_csv(file_path)
    df['Game'] = df['Game'].apply(lambda x: replace_team_names(x, mapping_dict))
    df.to_csv(file_path, index=False)

# Directory containing the raw data
input_directory = "c:/Users/luken/Desktop/Betting_Model_24_25/Raw_games_training"

# Collect all file paths from the directory
file_paths = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith(".csv")]

# Loop through the files and apply the mapping function
for file_path in file_paths:
    update_and_overwrite_file(file_path, mapping_dict)

print("Team names updated successfully in the 'Game' column of all files.")

Team names updated successfully in the 'Game' column of all files.


In [5]:
import pandas as pd
import os

# Define input and output directories
input_dir = "c:/Users/luken/Desktop/Betting_Model_24_25/Raw_games_training"  # Adjusted input directory path
output_dir = "c:/Users/luken/Desktop/Betting_Model_24_25/Selected_data"  # Updated output directory path

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Drop unwanted columns from the files
# Every stat has a "for", "against", and "for%" column; we'll drop columns with "%" in their name to reduce redundancy

# Get a list of all CSV files in the input directory
csv_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f)) and f.endswith('.csv')]

for file in csv_files:
    input_file_path = os.path.join(input_dir, file)
    output_file_path = os.path.join(output_dir, file)
    
    # Read the input CSV file
    df = pd.read_csv(input_file_path)
    
    # Identify columns to drop (those with '%' in the name)
    cols_to_drop = [col for col in df.columns if '%' in col]
    
    # Drop unnecessary columns
    df.drop(columns=cols_to_drop, inplace=True)
    for column in ['Unnamed: 2', 'Attendance', 'TOI']:
        if column in df.columns:  # Check if the column exists to avoid errors
            df.drop(column, axis=1, inplace=True)

    # Save the cleaned DataFrame to the output directory
    df.to_csv(output_file_path, index=False)

print(f"Processed files saved in {output_dir}")

Processed files saved in c:/Users/luken/Desktop/Betting_Model_24_25/Selected_data


In [6]:
import os
import pandas as pd

# Input and output directories
input_directory = "c:/Users/luken/Desktop/Betting_Model_24_25/Selected_data"  # Adjusted for your folder structure
base_output_directory = "c:/Users/luken/Desktop/Betting_Model_24_25/Sorted_data"  # Adjusted for your folder structure

# Each file represents all the games from the past year. Each unique game occurs twice:
# once for the home team and once from the away team's perspective.
# The "Team" column shows which team's perspective the stats are from.

# Iterate through the files and create new files for each team's games for each year
for file_name in os.listdir(input_directory):
    if file_name.endswith(".csv"):
        # Extract the year from the filename
        year = file_name.split('_games.csv')[0]

        # Create a directory for the year if it doesn't exist
        year_directory = os.path.join(base_output_directory, year)
        if not os.path.exists(year_directory):
            os.makedirs(year_directory)

        # Load the CSV file
        file_path = os.path.join(input_directory, file_name)
        df = pd.read_csv(file_path)

        # Extract details about the game (date, teams, scores)
        pattern = r'(\d{4}-\d{2}-\d{2}) - ([\w\s]+) (\d+), ([\w\s]+) (\d+)'
        df[['Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore']] = df['Game'].str.extract(pattern)

        # Determine the home team's result
        df['HomeResult'] = 'Draw'
        df.loc[df['HomeScore'] > df['AwayScore'], 'HomeResult'] = 'Won'
        df.loc[df['HomeScore'] < df['AwayScore'], 'HomeResult'] = 'Lost'

        # Create subsets for each team
        team_subsets = {team: df[df['Team'] == team] for team in df['Team'].unique()}

        # Save each team's data into a separate file within the year's directory
        for team, subset in team_subsets.items():
            team_file_name = f"{team}.csv"
            team_file_path = os.path.join(year_directory, team_file_name)
            subset.to_csv(team_file_path, index=False)

print(f"Files successfully sorted into {base_output_directory}")

Files successfully sorted into c:/Users/luken/Desktop/Betting_Model_24_25/Sorted_data


In [7]:
#loop through the files for each team in each year to calculate the number of days of rest each team had going into that game by taking the difference between the number of days between each game (time diff) minus 1 (days of rest)
import os
import pandas as pd

# Root directory containing sorted data
root_dir = "c:/Users/luken/Desktop/Betting_Model_24_25/Sorted_data"  # Adjusted for your folder structure

# List all subdirectories inside the root directory
subfolders = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

# Process each subfolder
for subfolder in subfolders:
    dir_path = os.path.join(root_dir, subfolder)
    
    # List all CSV files in the subdirectory
    all_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f)) and f.endswith('.csv')]

    # Process each file in the subdirectory
    for file in all_files:
        file_path = os.path.join(dir_path, file)
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the "Date" column exists
        if 'Date' in df.columns:
            date_col = 'Date'
        else:
            print(f"Cannot find 'Date' column in file {file} inside {subfolder}. Skipping...")
            continue
        
        # Convert the "Date" column to datetime
        df[date_col] = pd.to_datetime(df[date_col])
        
        # Calculate the difference between consecutive game dates
        df['time_diff'] = df[date_col].diff()
        
        # Calculate days of rest and drop the temporary "time_diff" column
        df['days_of_rest'] = df['time_diff'].dt.days - 1
        df.drop('time_diff', axis=1, inplace=True)
        
        # Save the updated DataFrame back to the same file
        df.to_csv(file_path, index=False)

        print(f"Processed {file} inside {subfolder}")

Processed ANA.csv inside 21_22_games_raw.csv
Processed ARI.csv inside 21_22_games_raw.csv
Processed BOS.csv inside 21_22_games_raw.csv
Processed BUF.csv inside 21_22_games_raw.csv
Processed CAR.csv inside 21_22_games_raw.csv
Processed CBJ.csv inside 21_22_games_raw.csv
Processed CGY.csv inside 21_22_games_raw.csv
Processed CHI.csv inside 21_22_games_raw.csv
Processed COL.csv inside 21_22_games_raw.csv
Processed DAL.csv inside 21_22_games_raw.csv
Processed DET.csv inside 21_22_games_raw.csv
Processed EDM.csv inside 21_22_games_raw.csv
Processed FLA.csv inside 21_22_games_raw.csv
Processed LA.csv inside 21_22_games_raw.csv
Processed MIN.csv inside 21_22_games_raw.csv
Processed MTL.csv inside 21_22_games_raw.csv
Processed NJ.csv inside 21_22_games_raw.csv
Processed NSH.csv inside 21_22_games_raw.csv
Processed NYI.csv inside 21_22_games_raw.csv
Processed NYR.csv inside 21_22_games_raw.csv
Processed OTT.csv inside 21_22_games_raw.csv
Processed PHI.csv inside 21_22_games_raw.csv
Processed PI

In [8]:
# Function to calculate the rolling average of the 10 games prior to each game
def calculate_rolling_average(df, columns, window=10):
    rolling_df = df[columns].rolling(window=window, min_periods=10).mean().shift(1)
    return rolling_df

# Base directory paths
base_path = "c:/Users/luken/Desktop/Betting_Model_24_25/Sorted_data"  # Input folder containing sorted data
average_data_path = "c:/Users/luken/Desktop/Betting_Model_24_25/Average_data"  # Output folder for processed data
os.makedirs(average_data_path, exist_ok=True)  # Create the output directory if it doesn't exist

# Process each year folder
for year_folder in os.listdir(base_path):
    year_path = os.path.join(base_path, year_folder)

    if os.path.isdir(year_path):  # Ensure it's a directory
        # Create a folder for the current year's averaged data
        year_avg_folder_path = os.path.join(average_data_path, f'avg_{year_folder}')
        os.makedirs(year_avg_folder_path, exist_ok=True)

        # Process each team file in the year folder
        for team_file in os.listdir(year_path):
            if team_file.endswith('.csv'):  # Process only CSV files
                team_path = os.path.join(year_path, team_file)
                df = pd.read_csv(team_path)
                
                # Select columns that should not be averaged (non-numeric, scores, and days of rest)
                selected_columns = [
                    'Game', 'Team', 'Date', 'AwayTeam', 'AwayScore',
                    'HomeTeam', 'HomeScore', 'HomeResult', 'days_of_rest'
                ]
                # Identify numeric columns to calculate rolling averages
                other_columns = df.columns.difference(selected_columns)

                # Calculate rolling averages for numeric columns
                rolling_df = calculate_rolling_average(df, other_columns)

                # Combine non-averaged columns with the averaged columns
                combined_df = pd.concat([df[selected_columns], rolling_df], axis=1)

                # Save the processed file in the corresponding year's folder
                processed_file_path = os.path.join(year_avg_folder_path, f'{team_file}')
                combined_df.to_csv(processed_file_path, index=False)

                print(f"Processed file saved to: {processed_file_path}")

Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\ANA.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\ARI.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\BOS.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\BUF.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\CAR.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\CBJ.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\CGY.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games_raw.csv\CHI.csv
Processed file saved to: c:/Users/luken/Desktop/Betting_Model_24_25/Average_data\avg_21_22_games

In [10]:
# Directory containing raw data
directory_path = "c:/Users/luken/Desktop/Betting_Model_24_25/Raw_games_training"  # Adjusted to your folder structure

# Create the initial empty DataFrame to aggregate all games
master_df = pd.DataFrame()

# Loop through all files in the raw data folder and add games to the master DataFrame
for file_name in os.listdir(directory_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        if 'Game' in df.columns:
            master_df = pd.concat([master_df, df[['Game']]])

# Extract the information from the string in the 'Game' column
pattern = r'(\d{4}-\d{2}-\d{2}) - ([\w\s]+) (\d+), ([\w\s]+) (\d+)'
master_df[['Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore']] = master_df['Game'].str.extract(pattern)

# Determine the result of the game for the home team
master_df['HomeResult'] = 'Draw'
master_df.loc[master_df['HomeScore'].astype(int) > master_df['AwayScore'].astype(int), 'HomeResult'] = 'Won'
master_df.loc[master_df['HomeScore'].astype(int) < master_df['AwayScore'].astype(int), 'HomeResult'] = 'Lost'

# Filter to keep only unique games (every second row)
filt_df = master_df.iloc[1::2]  # Assumes every unique game appears twice

# Display the first 20 rows of the filtered DataFrame
print(filt_df.head(20))

                         Game        Date AwayTeam AwayScore HomeTeam  \
1    2021-10-12 - PIT 6, TB 2  2021-10-12      PIT         6       TB   
3   2021-10-12 - SEA 3, VGK 4  2021-10-12      SEA         3      VGK   
5   2021-10-13 - MTL 1, TOR 2  2021-10-13      MTL         1      TOR   
7   2021-10-13 - NYR 1, WSH 5  2021-10-13      NYR         1      WSH   
9   2021-10-13 - CHI 2, COL 4  2021-10-13      CHI         2      COL   
11  2021-10-13 - VAN 2, EDM 3  2021-10-13      VAN         2      EDM   
13  2021-10-13 - WPG 1, ANA 4  2021-10-13      WPG         1      ANA   
15  2021-10-14 - MTL 1, BUF 5  2021-10-14      MTL         1      BUF   
17  2021-10-14 - TOR 2, OTT 3  2021-10-14      TOR         2      OTT   
19   2021-10-14 - TB 7, DET 6  2021-10-14       TB         7      DET   
21  2021-10-14 - PIT 4, FLA 5  2021-10-14      PIT         4      FLA   
23  2021-10-14 - DAL 3, NYR 2  2021-10-14      DAL         3      NYR   
25  2021-10-14 - NYI 3, CAR 6  2021-10-14      NYI 

In [17]:
import os

# Path to the directory containing the folders/files
base_directory = 'c:/Users/luken/Desktop/Betting_Model_24_25/Average_data'

# Iterate over the items in the directory
for item in os.listdir(base_directory):
    item_path = os.path.join(base_directory, item)
    
    # Check if the item is a directory and incorrectly named
    if os.path.isdir(item_path) and '.csv' in item:
        # Construct the new name without '.csv'
        new_name = item.replace('.csv', '')
        new_path = os.path.join(base_directory, new_name)
        
        # Rename the folder
        os.rename(item_path, new_path)
        print(f"Renamed: {item} -> {new_name}")

Renamed: avg_21_22_games_raw.csv -> avg_21_22_games_raw
Renamed: avg_22_23_games_raw.csv -> avg_22_23_games_raw
Renamed: avg_23_24_games_raw.csv -> avg_23_24_games_raw


In [20]:
year_directories = [
    r'C:\Users\luken\Desktop\Betting_Model_24_25\Average_data\avg_21_22_games_raw',
    r'C:\Users\luken\Desktop\Betting_Model_24_25\Average_data\avg_22_23_games_raw',
    r'C:\Users\luken\Desktop\Betting_Model_24_25\Average_data\avg_23_24_games_raw'
]
combined_directory = 'stats'
os.makedirs(combined_directory, exist_ok=True)


team_dfs = {}

for directory in year_directories:
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            team_name = filename.replace('_games.csv', '')
            file_path = os.path.join(directory, filename)

            df = pd.read_csv(file_path)
            cols_to_drop = ['Date','AwayTeam','AwayScore','HomeTeam','HomeScore','HomeResult']
            df.drop(columns=cols_to_drop, inplace=True)
            if team_name in team_dfs:
                team_dfs[team_name] = pd.concat([team_dfs[team_name], df])
            else:
                team_dfs[team_name] = df

for team_name, df in team_dfs.items():
    output_path = os.path.join(combined_directory, f'{team_name}')
    df.to_csv(output_path, index=False)

In [21]:
import os
import pandas as pd

# DataFrame containing all the games
games_df = filt_df

# Create the function to add home team stats
def add_home_team_stats(row):
    team_name = row['HomeTeam']
    file_path = f'stats/{team_name}.csv'
    
    # Check if the file exists to avoid runtime errors
    if not os.path.exists(file_path):
        print(f"Stats file for {team_name} not found: {file_path}")
        return row

    # Load the team's stats
    team_stats_df = pd.read_csv(file_path)

    # Match the game row using the 'Game' column
    game_row = team_stats_df[team_stats_df['Game'] == row['Game']]
    
    if not game_row.empty:
        # Add a prefix to home team stats and merge with the current row
        team_stats = game_row.add_prefix('h_').iloc[0]
        team_stats.rename({'h_Game': 'Game'}, inplace=True)  # Rename back 'Game' to original
        for col in team_stats.index:
            row[col] = team_stats[col]
    return row

# Process each row in the games DataFrame and collect updated rows
updated_rows = []

for index, row in games_df.iterrows():
    updated_row = add_home_team_stats(row)  # Update the row with home team stats
    updated_rows.append(updated_row)

# Create a new DataFrame with the updated rows
hgames_df = pd.DataFrame(updated_rows)

# Display the first few rows of the resulting DataFrame
print(hgames_df.head())

                        Game        Date AwayTeam AwayScore HomeTeam  \
1   2021-10-12 - PIT 6, TB 2  2021-10-12      PIT         6       TB   
3  2021-10-12 - SEA 3, VGK 4  2021-10-12      SEA         3      VGK   
5  2021-10-13 - MTL 1, TOR 2  2021-10-13      MTL         1      TOR   
7  2021-10-13 - NYR 1, WSH 5  2021-10-13      NYR         1      WSH   
9  2021-10-13 - CHI 2, COL 4  2021-10-13      CHI         2      COL   

  HomeScore HomeResult h_Team  h_days_of_rest  h_CA  ...  h_MDGF  h_MDSA  \
1         2       Lost     TB             NaN   NaN  ...     NaN     NaN   
3         4        Won    VGK             NaN   NaN  ...     NaN     NaN   
5         2        Won    TOR             NaN   NaN  ...     NaN     NaN   
7         5        Won    WSH             NaN   NaN  ...     NaN     NaN   
9         4        Won    COL             NaN   NaN  ...     NaN     NaN   

   h_MDSF  h_PDO  h_SA  h_SCA  h_SCF  h_SF  h_xGA  h_xGF  
1     NaN    NaN   NaN    NaN    NaN   NaN    NaN  

In [22]:
import os
import pandas as pd

# DataFrame containing all the games
games_df = filt_df

# Create the function to add away team stats
def add_away_team_stats(row):
    team_name = row['AwayTeam']
    file_path = f'stats/{team_name}.csv'
    
    # Check if the file exists to avoid runtime errors
    if not os.path.exists(file_path):
        print(f"Stats file for {team_name} not found: {file_path}")
        return row

    # Load the team's stats
    team_stats_df = pd.read_csv(file_path)

    # Match the game row using the 'Game' column
    game_row = team_stats_df[team_stats_df['Game'] == row['Game']]
    
    if not game_row.empty:
        # Add a prefix to away team stats and merge with the current row
        team_stats = game_row.add_prefix('a_').iloc[0]
        team_stats.rename({'a_Game': 'Game'}, inplace=True)  # Rename back 'Game' to original
        for col in team_stats.index:
            row[col] = team_stats[col]
    return row

# Process each row in the games DataFrame and collect updated rows
updated_rows = []

for index, row in games_df.iterrows():
    updated_row = add_away_team_stats(row)  # Update the row with away team stats
    updated_rows.append(updated_row)

# Create a new DataFrame with the updated rows
agames_df = pd.DataFrame(updated_rows)

# Display the first few rows of the resulting DataFrame
print(agames_df.head())

                        Game        Date AwayTeam AwayScore HomeTeam  \
1   2021-10-12 - PIT 6, TB 2  2021-10-12      PIT         6       TB   
3  2021-10-12 - SEA 3, VGK 4  2021-10-12      SEA         3      VGK   
5  2021-10-13 - MTL 1, TOR 2  2021-10-13      MTL         1      TOR   
7  2021-10-13 - NYR 1, WSH 5  2021-10-13      NYR         1      WSH   
9  2021-10-13 - CHI 2, COL 4  2021-10-13      CHI         2      COL   

  HomeScore HomeResult a_Team  a_days_of_rest  a_CA  ...  a_MDGF  a_MDSA  \
1         2       Lost    PIT             NaN   NaN  ...     NaN     NaN   
3         4        Won    SEA             NaN   NaN  ...     NaN     NaN   
5         2        Won    MTL             NaN   NaN  ...     NaN     NaN   
7         5        Won    NYR             NaN   NaN  ...     NaN     NaN   
9         4        Won    CHI             NaN   NaN  ...     NaN     NaN   

   a_MDSF  a_PDO  a_SA  a_SCA  a_SCF  a_SF  a_xGA  a_xGF  
1     NaN    NaN   NaN    NaN    NaN   NaN    NaN  

In [23]:
#merge the two dataframes to create one master dataframe with all the away and home team stats, then save the datraframe to the a csv file
import pandas as pd
common_columns = ['Game', 'Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'HomeResult']

merged_df = pd.merge(agames_df, hgames_df, on=common_columns, how='outer')
df2 = merged_df.dropna()
cols_to_drop = ['a_Team','h_Team']
df2.drop(columns=cols_to_drop, inplace=True)

df2.to_csv('training_set.csv', index=False)
#the training set is now stored in "training_set.csv"

C:\Users\luken\AppData\Local\Temp\ipykernel_21768\3605726190.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=cols_to_drop, inplace=True)
